In [1]:
import numpy as np
from skimage import morphology, color, filters, exposure
from skimage.io import imread

def initialize_membership(data, cluster_n):
    # Initialize membership matrix with random values
    U = np.random.rand(data.shape[0], cluster_n)
    # Normalize each row to sum to 1
    U /= np.sum(U, axis=1, keepdims=True)
    return U

def frfcm_c(data, cluster_n, radius, w_size, options=None):
    if options is None:
        options = [2, 100, 1e-5, 0]

    expo, max_iter, min_impro, display = options

    if expo <= 1:
        raise ValueError("The exponent should be greater than 1!")

    # Step 1: Morphological reconstruction
    selem = morphology.disk(radius)
    reconstructed_data = morphology.reconstruction(data, data, method='dilation', selem=selem)

    # Convert to grayscale if it's not already
    if reconstructed_data.ndim == 3 and reconstructed_data.shape[2] == 3:
        grayscale_data = color.rgb2gray(reconstructed_data)
    else:
        grayscale_data = reconstructed_data

    # Step 2: Calculate histogram
    hist, bin_edges = np.histogram(grayscale_data.ravel(), bins=256, range=(0, 1))
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    data_hist = np.vstack((bin_centers, hist)).T

    # Initialize membership
    U = initialize_membership(data_hist, cluster_n)

    # Main loop
    for i in range(max_iter):
        mf = np.power(U, expo)

        # Compute cluster centers
        center = np.empty((cluster_n, data_hist.shape[1]))
        for j in range(cluster_n):
            numerator = np.sum(data_hist * mf[:, j][:, None], axis=0)
            denominator = np.sum(mf[:, j]) + np.finfo(np.float64).eps
            center[j, :] = numerator / denominator

        # Compute distances and update objective function
        dist = np.linalg.norm(data_hist[:, None, :] - center[None, :, :], axis=2)
        obj_fcn = np.sum(dist ** 2 * mf)

        # Update membership matrix
        U_new = np.power(dist, -2 / (expo - 1))
        U_new /= np.sum(U_new, axis=1)[:, None] + np.finfo(np.float64).eps

        # Check for convergence
        if i > 0 and np.max(np.abs(U_new - U)) < min_impro:
            break
        U = U_new

    # Apply median filter to membership matrix
    U_filtered = filters.median(U.reshape(-1, 256), selem=morphology.disk(w_size)).ravel()
    U_normalized = U_filtered / (np.sum(U_filtered) + np.finfo(np.float64).eps)

    # The cluster centers in the context of a histogram are just the bin centers weighted by the membership
    center_values = np.sum(center[:, 0][:, None] * U_normalized, axis=0)

    return center_values, U_normalized, obj_fcn, i
